In [ ]:
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
mpl.rc('font', family='serif', size=11)
mpl.rc('savefig', bbox='tight')

import dtw
import worg

make_fname_safe = lambda s: s.lower().replace(' ', '-')

In [ ]:
T = 50
seed = 424242
verbose = True
f = 90 * (1.01**np.arange(T))  # 1% growth
g0 = np.zeros(T, dtype='f8')    # zero production
N = np.asarray(np.ceil(4*(1.01)**np.arange(T)), dtype=int)

DTW
======

In [ ]:
def display_dwt(dist, cost, path, xname, yname, offset=2016, vmax=None):
    fig = plt.figure(figsize=(8, 6))
    extent = (offset, offset+cost.shape[1], offset, offset+cost.shape[0])
    plt.imshow(cost[::-1], cmap='viridis', extent=extent, vmin=0.0, vmax=vmax)
    u = offset + np.concatenate([path, np.array([[cost.shape[1]], [cost.shape[0]]])], axis=1)
    plt.plot(u[1], u[0], 'w-')
    plt.axis(extent)
    cb = plt.colorbar()
    cb.set_label('Cost [GWe]', rotation=-90, va='bottom')
    plt.xlabel('time [year]')
    plt.ylabel('time [year]')
    fname = 'cost-{0}-to-{1}'.format(make_fname_safe(xname), make_fname_safe(yname))
    plt.savefig(fname + '.png')
    plt.savefig(fname + '.eps')
    print('Warping between {0} and {1}:'.format(xname, yname))
    print('  Distance is ', dist)

In [ ]:
p = np.empty(T, dtype='f8')
p[:T//2] = f[:T//2] * 0.95
p[T//2:] = f[T//2:] * 1.05
dist, cost, path = dtw.dtw(f[:,np.newaxis], p[:,np.newaxis])
display_dwt(dist, cost, path, 'Demand', 'Production')

GP
=======

In [ ]:
def model_plot(t, y, yerr, tpred, mu, std, title=None, ymax=None):
    fig = plt.figure(figsize=(8, 8))
    #plt.errorbar(t, y, yerr, fmt='r.', label='traing data')
    plt.plot(t, y, 'r.', label='traing data')
    plt.plot(tpred, mu, 'k-', label='model')
    plt.fill_between(tpred, mu - 2*std, mu + 2*std, color='gray')
    ax = plt.axis()
    plt.axis([tpred[0], tpred[-1], 0.0, ymax or ax[3]])
    plt.legend(loc=0)
    plt.xlabel('time [year]')
    plt.ylabel('Power Production [GWe]')
    if title:
        plt.title(title)
    fbase = 'gwe-model-' + (title or '').lower().replace(' ', '-')
    plt.savefig(fbase + '.eps')
    plt.savefig(fbase + '.png')

In [ ]:
tol = 1e-1
q = np.empty(T, dtype='f8')
q[:T//2] = f[:T//2] * 0.9
q[T//2:] = f[T//2:] * 1.1
gp, x, y = worg.gp_gwe([N, N], [f, q], T, tol, N)
mu, cov = gp.predict(y, x[:T])
std = np.sqrt(np.diag(cov))
model_plot(x[:,0]+2016, y, 0.0, x[:T,0] + 2016, mu, std)

Optimizations
========

In [ ]:
def run_percent_growth(rate, f0=90.0, N0=4, T=T, seed=seed, verbose=verbose, **kwargs):
    r = 1.0 + (rate/100.0)
    tgrid = np.arange(T)
    f = f0 * (r**tgrid)
    N = np.asarray(np.ceil(N0*(r**tgrid)), dtype=int) + 1
    state = worg.optimize(f, N, seed=seed, verbose=verbose, **kwargs)
    return state

In [ ]:
%time state1all = run_percent_growth(1, MAX_S=20)

In [ ]:
%time state1sto = run_percent_growth(1, MAX_S=20, method_0='stochastic')

In [ ]:
%time state1inn = run_percent_growth(1, MAX_S=20, method_0='inner-prod')

In [ ]:
%time state0sto = run_percent_growth(0, MAX_S=20, method_0='stochastic')

In [ ]:
%time state2sto = run_percent_growth(2, MAX_S=20, method_0='stochastic')

In [ ]:
%time state2all = run_percent_growth(2, MAX_S=20, method_0='all')

In [ ]:
%time state2inn = run_percent_growth(2, MAX_S=20, method_0='inner-prod')

In [ ]:
%time state0all = run_percent_growth(0, MAX_S=20, method_0='all')

In [ ]:
%time state0inn = run_percent_growth(0, MAX_S=20, method_0='inner-prod')

In [ ]:
def convergence_plot(states, linestyles, labels, r):
    for state, ls, label in zip(states, linestyles, labels):
        x = np.arange(1, state['s'] + 1)
        D_s = state['D_s']
        y = np.fromiter((min(D_s[:i]) for i in x), dtype='f8', count=len(x))
        plt.semilogy(x, y, linestyle=ls, color='gray', label=label)
        smask = np.fromiter((w == 'stochastic' for w in state['winner_s']), 
                            dtype='bool', count=len(x))
        if np.any(smask):
            plt.semilogy(x[smask], y[smask], linestyle='None', marker='$S$')
        imask = np.fromiter((w == 'inner-prod' for w in state['winner_s']), 
                            dtype='bool', count=len(x))
        if np.any(imask):
            plt.semilogy(x[imask], y[imask], linestyle='None', marker='$I$')
        amask = np.fromiter((w == 'all' for w in state['winner_s']), 
                            dtype='bool', count=len(x))
        if np.any(amask):
            plt.semilogy(x[amask], y[amask], linestyle='None', marker='$A$')
    plt.axis([1, 20, 0.1, 100])
    plt.grid()
    plt.legend(loc=0)
    plt.xlabel('Simulation $s$')
    plt.ylabel('Minimum Distance, $d(f, g_s)$')
    plt.title('{}% Growth'.format(r))
    fname = 'converge-{}per'.format(r)
    plt.savefig(fname + '.eps')
    plt.savefig(fname + '.png')

In [ ]:
linestyles = ['--', 'dotted', '-']
labels = ['stochastic', 'inner-prod', 'all']

In [ ]:
convergence_plot([state0sto, state0inn, state0all], linestyles, labels, 0)

In [ ]:
convergence_plot([state1sto, state1inn, state1all], linestyles, labels, 1)

In [ ]:
convergence_plot([state2sto, state2inn, state2all], linestyles, labels, 2)

In [ ]:
def demand_product(states, title):
    rates = [0, 1, 2]
    colors = ['k', 'green', 'purple']
    for state, color, rate in zip(states, colors, rates):
        f = state['f']
        g = state['G'][0]
        t = 2016 + np.arange(len(g))
        plt.plot(t, f, linestyle='--', color=color)
        plt.plot(t, g, linestyle='-', color=color, label='{}% growth'.format(rate))
    plt.legend(loc=0)
    plt.axis([2016, 2065, 0, 250])
    plt.xlabel('time [year]')
    plt.ylabel('Power [GWe]')
    plt.title(title + ' Method')
    fname = 'demand-product-{}'.format(title.lower())
    plt.savefig(fname + '.eps')
    plt.savefig(fname + '.png')

In [ ]:
demand_product([state0sto, state1sto, state2sto], 'Stochastic')

In [ ]:
demand_product([state0inn, state1inn, state2inn], 'Inner Product')

In [ ]:
demand_product([state0all, state1all, state2all], 'All')